# Applying the optimization to any number fo n features

The Iris data set is a well known small sample of petal and sepal measurements, that allow the distinction of three different varieties. As such, it fulfills the requirements of being reasonably small with a very small k for this algorithm.

In [1]:
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt
from sklearn.metrics import DistanceMetric
from scipy.spatial import distance as dist
import pandas as pd # feels slightly more flexible than np for parsing matrices
import numpy as np # although gurobi does funny things with data frames so lets use an array instead

## Reading the data, defining the number of features and clusters

In [2]:
def read_data(filename):
    points = pd.read_csv(filename)
    points = points.iloc[:, :-1] # last column is species name
    return points[:-1]

In [3]:
filename = "project/data_sets/iris.data"
points = np.array(read_data(filename))
k = 3

## Computing the LP

In [ ]:
# distance function
distances = DistanceMetric.get_metric('euclidean')

# we need all distances as possible radii
radii = distances.pairwise(points)

# model
m = gp.Model("kmsr")

# variable
y = m.addVars(len(points), len(points), vtype=GRB.BINARY, name="Y") # for dummies (me): adds a 10x10 grid of binary variables y_ij that can be either 0 or 1, which will be used to check which clusters overlap a given point

# objective:
m.setObjective(gp.quicksum([y[i, j] * radii[i][j] for i in range(len(radii)) for j in range(len(radii[i]))]),  GRB.MINIMIZE) # should be the minimization of the sum of active y_ij times their respective radii

# constraints:

# every point is covered by atleast one center/radius y_ij
for n in range(len(points)):
        m.addConstr(gp.quicksum([y[i, j] for j in range(len(radii)) for i in range (len(radii)) if radii[i][j] >= dist.euclidean(points[n], points[i])]) >= 1, "every_point_covered") 
"""for every point we enforce that the sum of all y_ij that represent a center/radius combination where the distance from the point to that center is 
    close enough to the point for the radius to cover it is atleast 1. because then, there is always a y_ij active for every point that covers it"""

# exactly k centers open in total (not per row of y, so we don't get the sum of 10 different solutions)
m.addConstr(gp.quicksum([y[i, j] for i in range (len(radii)) for j in range(len(radii[i]))]) == k, "select_k_Centers")

# do the thing
m.optimize()

In [ ]:
# currently does not work as it's own function, because .x is not recognized by the gurobi.var object as the attribute value (for some reason)

final_centers = [points[i] for i in range(len(points)) for j in range(len(radii[i])) if y[i, j].x == 1]
final_radii = [radii[i][j] for i in range(len(radii)) for j in range(len(radii[i])) if y[i, j].x == 1]

print("Optimal centers:")
for i in range(len(final_centers)): 
    print(f"Center {i} at point {final_centers[i]} with radius {final_radii[i]}")

## Simple Example

In [ ]:
points = np.array([[1, 1, 1], [1, 1, 0], [1, 0, 1], 
          [5, 5, 5], [5, 5, 4], [5, 4, 5],
          [10, 10, 10], [10, 10, 9], [10, 9, 10]], dtype=np.number)
k = 3

C:\Users\Mattis\AppData\Local\Temp\ipykernel_12536\3459906729.py:1: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  points = np.array([[1, 1, 1], [1, 1, 0], [1, 0, 1],


In [7]:
# repeat entire section because of gurobi weirdness. have to work on that

# distance function
distances = DistanceMetric.get_metric('euclidean')

# we need all distances as possible radii
radii = distances.pairwise(points)

# model
m = gp.Model("k_msr")

# variable
y = m.addVars(len(points), len(points), vtype=GRB.BINARY, name="Y") # for dummies (me): adds a 10x10 grid of binary variables y_ij that can be either 0 or 1, which will be used to check which clusters overlap a given point

# objective:
m.setObjective(gp.quicksum([y[i, j] * radii[i][j] for i in range(len(radii)) for j in range(len(radii[i]))]),  GRB.MINIMIZE) # should be the minimization of the sum of active y_ij times their respective radii

# constraints:

# every point is covered by atleast one center/radius y_ij
for n in range(len(points)):
        m.addConstr(gp.quicksum([y[i, j] for j in range(len(radii)) for i in range (len(radii)) if radii[i][j] >= dist.euclidean(points[n], points[i])]) >= 1, "every_point_covered") 
"""for every point we enforce that the sum of all y_ij that represent a center/radius combination where the distance from the point to that center is 
    close enough to the point for the radius to cover it is atleast 1. because then, there is always a y_ij active for every point that covers it"""

# exactly k centers open in total (not per row of y, so we don't get the sum of 10 different solutions)
m.addConstr(gp.quicksum([y[i, j] for i in range (len(radii)) for j in range(len(radii[i]))]) == k, "select_k_Centers")

# do the thing
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10 rows, 81 columns and 495 nonzeros
Model fingerprint: 0x10ce8c8c
Variable types: 0 continuous, 81 integer (81 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 25.8620758
Presolve added 1 rows and 0 columns
Presolve removed 0 rows and 31 columns
Presolve time: 0.00s
Presolved: 11 rows, 50 columns, 249 nonzeros
Variable types: 0 continuous, 50 integer (32 binary)

Root relaxation: objective 3.000000e+00, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Bes

In [8]:
# see above comment

final_centers = [points[i] for i in range(len(points)) for j in range(len(radii[i])) if y[i, j].x == 1]
final_radii = [radii[i][j] for i in range(len(radii)) for j in range(len(radii[i])) if y[i, j].x == 1]

print("Optimal centers:")
for i in range(len(final_centers)): 
    print(f"Center {i} at point {final_centers[i]} with radius {final_radii[i]}")

Optimal centers:
Center 0 at point [1. 1. 1.] with radius 1.0
Center 1 at point [5. 5. 5.] with radius 1.0
Center 2 at point [10. 10. 10.] with radius 1.0
